## Esse notebook apresenta as hipóteses

**As hipóteses a serem testadas são:**

1 - Jogos disputados em casa têm uma maior probabilidade de terminar com vitória para o time da casa

2 - Times com mais gols no campeonato tendem a marcar um gol primeiro

3 - Quanto mais cartões o time de fora toma, mais chances o time da casa tem de ganhar.

**Tutorial de como rodar esse notebook:**

1 - Ter as tabelas colunas-matches.csv e colunas-teams.csv dentro da pasta pre-processamento

2 - As bibliotecas necessárias já estão sendo importadas no código, mas se, por algum motivo, não funcionar, instale as bibliotecas no seu computador através do comando **pip install (nome da biblioteca)**

3 - Ter o Python instalado

## Hipótese 1

Hipótese: Jogos disputados em casa têm uma maior probabilidade de terminar com vitória para o time da casa

Teste: Comparar vitória dos times com os times que jogaram em casa nas vitórias. Segue o código de criação do gráfico.


**Importação**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Carregamento das tabelas**

In [2]:
# Carregar os dados
df = pd.read_csv("../pre-processamento/colunas-matches.csv")
dff = pd.read_csv("../pre-processamento/colunas-teams.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../pre-processamento/colunas-matches.csv'

**Código**

In [ ]:
# Selecionar as colunas necessárias
colunas_desejadas = ['home_team_name', 'away_team_name', 'home_team_goal_count', 'away_team_goal_count']
df_selecionado = df[colunas_desejadas]

In [ ]:
# Criar uma coluna para indicar onde foi a vitória
df_selecionado['resultado'] = df_selecionado.apply(
    lambda row: 'Casa' if row['home_team_goal_count'] > row['away_team_goal_count'] 
    else 'Fora' if row['home_team_goal_count'] < row['away_team_goal_count']
    else 'Empate', axis=1
)

In [ ]:

# Filtrar apenas os jogos que não terminaram empatados
df_vitorias = df_selecionado[df_selecionado['resultado'] != 'Empate']

# Contar as vitórias em casa e fora
contagem_vitorias = df_vitorias['resultado'].value_counts()

In [ ]:

# Plotar o gráfico
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_vitorias.index, y=contagem_vitorias.values)
plt.title('Número de Vitórias em Casa e Fora')
plt.xlabel('Local da Vitória')
plt.ylabel('Número de Vitórias')
plt.show()

Dessa forma, Através do gráfico acima, é comprovado que os times que jogam em casa tendem a vencer mais do que os times que jogam fora, visto que o time da casa tem 41 vitórias no campeonato e o time de fora tem 26

## Hipótese 2

**Hipótese:** Times com mais gols no campeonato tendem a marcar um gol primeiro

**Teste:** Comparar os times com mais gols e os times que marcaram primeiro. Segue a baixo a criação do gráfico que testa essa hipótese.

In [ ]:
# Somar os gols marcados por cada time (em casa e fora)
gols_em_casa = df.groupby('home_team_name')['home_team_goal_count'].sum()
gols_fora = df.groupby('away_team_name')['away_team_goal_count'].sum()
total_gols = gols_em_casa.add(gols_fora, fill_value=0).reset_index(name='total_gols')

In [ ]:
# Somar as vezes que cada time marcou primeiro (em casa e fora)
primeiro_gol_em_casa = dff.groupby('team_name')['first_team_to_score_count_home'].sum()
primeiro_gol_fora = dff.groupby('team_name')['first_team_to_score_count_away'].sum()
total_primeiros_gols = primeiro_gol_em_casa.add(primeiro_gol_fora, fill_value=0).reset_index(name='total_primeiros_gols')

In [ ]:

# Unir os dois DataFrames pelo nome do time
df_combinado = pd.merge(total_gols, total_primeiros_gols, left_on='home_team_name', right_on='team_name', how='outer')

# Ordenar o DataFrame pelo total de gols em ordem decrescente
df_gols_ordenado = df_combinado.sort_values(by='total_gols', ascending=False)

# Ordenar o DataFrame pelo total de primeiros gols em ordem decrescente
df_primeiros_gols_ordenado = df_combinado.sort_values(by='total_primeiros_gols', ascending=False)

In [ ]:

# Plotar o gráfico do total de gols
plt.figure(figsize=(14, 8))
sns.barplot(x='home_team_name', y='total_gols', data=df_gols_ordenado, color='blue', label='Total de Gols')
plt.title('Total de Gols por Time em Ordem Decrescente')
plt.xlabel('Times')
plt.ylabel('Quantidade')
plt.xticks(rotation=90)
plt.legend()
plt.show()


In [ ]:

# Plotar o gráfico dos primeiros gols
plt.figure(figsize=(14, 8))
sns.barplot(x='team_name', y='total_primeiros_gols', data=df_primeiros_gols_ordenado, color='red', label='Primeiros Gols')
plt.title('Primeiros Gols por Time em Ordem Decrescente')
plt.xlabel('Times')
plt.ylabel('Quantidade')
plt.xticks(rotation=90)
plt.legend()
plt.show()

É possível ver que o time que mais marcou gols (botafogo) é o mesmo que o time que mais marcou gols primeiro. Entretanto, temos times como o fluminense ou flamengo que fogem dessa lógica. Portanto, essa hipótese não está 100% correta

## Hipótese 3
**Hipótese:** Quanto mais cartões o time de fora toma, mais chances o time da casa tem de ganhar.
**Teste:** Criar gráficos que mostrem quem leva mais cartões (o time da casa ou o time de fora) e ver qual deles mais vence. Segue a criação dos gráficos que testam a hipótese:

In [ ]:
# Criar uma coluna para comparar quem recebeu mais cartões (Casa ou Fora)
dff['cartoes_mais'] = dff.apply(lambda row: 'Casa' if row['cards_per_match_home'] > row['cards_per_match_away'] 
                                else 'Fora', axis=1)

# Contar as ocorrências de quem recebeu mais cartões
cartoes_comparacao = dff['cartoes_mais'].value_counts()

In [ ]:
# Plotar o gráfico de comparação de cartões
plt.figure(figsize=(10, 6))
sns.barplot(x=cartoes_comparacao.index, y=cartoes_comparacao.values, palette='coolwarm')
plt.title('Comparação de Cartões Recebidos (Casa x Fora)')
plt.xlabel('Quem Recebeu Mais Cartões')
plt.ylabel('Número de Partidas')
plt.show()

In [ ]:
# Plotar o gráfico
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_vitorias.index, y=contagem_vitorias.values, palette="muted")
plt.title('Número de Vitórias em Casa e Fora')
plt.xlabel('Local da Vitória')
plt.ylabel('Número de Vitórias')
plt.show()


Levando em conta os gráficos a cima, comprovamos que o time de fora costuma levar mais cartões e, ao mesmo tempo, perder mais partidas. Isso não prova diretamente que quanto mais cartões, mais perto da derrota, mas mostra que o time que leva menos carões tende a terminar a partida com a vitória.